In [19]:
import ipywidgets as widgets
from IPython.display import display, clear_output, HTML
import pandas as pd
from datetime import datetime
import json

class MehrfachDatenEingabe:
    def __init__(self):
        # Dictionary für mehrere Datensätze
        self.alle_datensaetze = {}
        self.aktueller_datensatz = None
        
        # CSS zum Verstecken der Spinner-Pfeile
        display(HTML("""
        <style>
            input[type=number]::-webkit-inner-spin-button,
            input[type=number]::-webkit-outer-spin-button {
                -webkit-appearance: none;
                margin: 0;
            }
            input[type=number] {
                -moz-appearance: textfield;
            }
        </style>
        """))
        
        # Datensatz-Verwaltung Widgets
        self.datensatz_name = widgets.Text(
            value='',
            placeholder='Name des neuen Datensatzes',
            description='Datensatz:',
            style={'description_width': '120px'}
        )
        
        self.datensatz_dropdown = widgets.Dropdown(
            options=['-- Neuer Datensatz --'],
            value='-- Neuer Datensatz --',
            description='Aktiver Datensatz:',
            style={'description_width': '120px'}
        )
        self.datensatz_dropdown.observe(self.datensatz_wechseln, names='value')
        
        self.neuer_datensatz_button = widgets.Button(
            description='Neuen Datensatz erstellen',
            button_style='primary',
            icon='plus'
        )
        self.neuer_datensatz_button.on_click(self.neuer_datensatz)
        
        # Eingabefelder - Verwende Text statt FloatText für keine Pfeile
        self.spannung_input = widgets.Text(
            value='',
            description='Spannung (V):',
            placeholder='Spannung in V',
            style={'description_width': '120px'}
        )
        
        self.stromstaerke_input = widgets.Text(
            value='',
            description='Stromstärke (mA):',
            placeholder='Stromstärke in mA',
            style={'description_width': '120px'}
        )
        
        self.drehzahl_input = widgets.Text(
            value='',
            description='Drehzahl (U/min):',
            placeholder='Drehzahl in U/min',
            style={'description_width': '120px'}
        )
        
        # Notiz-Feld für zusätzliche Informationen
        self.notiz_input = widgets.Text(
            value='',
            placeholder='Optionale Notiz',
            description='Notiz:',
            style={'description_width': '120px'}
        )
        
        # Buttons
        self.speichern_button = widgets.Button(
            description='Wert hinzufügen',
            button_style='success',
            icon='plus-circle',
            disabled=True
        )
        self.speichern_button.on_click(self.speichern_wert)
        
        self.anzeigen_button = widgets.Button(
            description='Datensatz anzeigen',
            button_style='info',
            icon='eye'
        )
        self.anzeigen_button.on_click(self.anzeigen_aktueller_datensatz)
        
        self.alle_anzeigen_button = widgets.Button(
            description='Übersicht',
            button_style='info',
            icon='list'
        )
        self.alle_anzeigen_button.on_click(self.alle_anzeigen)
        
        self.loeschen_wert_button = widgets.Button(
            description='Letzten Wert löschen',
            button_style='warning',
            icon='undo'
        )
        self.loeschen_wert_button.on_click(self.letzten_wert_loeschen)
        
        self.loeschen_datensatz_button = widgets.Button(
            description='Datensatz löschen',
            button_style='danger',
            icon='trash'
        )
        self.loeschen_datensatz_button.on_click(self.datensatz_loeschen)
        
        self.export_button = widgets.Button(
            description='Datensatz als CSV',
            button_style='primary',
            icon='download'
        )
        self.export_button.on_click(self.export_aktueller_datensatz)
        
        self.export_alle_button = widgets.Button(
            description='Alle als CSV',
            button_style='primary',
            icon='download'
        )
        self.export_alle_button.on_click(self.export_alle)
        
        # Speichern/Laden Buttons
        self.save_session_button = widgets.Button(
            description='Session speichern',
            button_style='',
            icon='save'
        )
        self.save_session_button.on_click(self.save_session)
        
        self.load_session_button = widgets.Button(
            description='Session laden',
            button_style='',
            icon='folder-open'
        )
        self.load_session_button.on_click(self.load_session)
        
        # Output-Bereich
        self.output = widgets.Output()
        
        # Layout erstellen
        self.layout = widgets.VBox([
            widgets.HTML('<h3>Multi-Datensatz Messdaten-Eingabe</h3>'),
            widgets.HBox([
                self.datensatz_name,
                self.neuer_datensatz_button
            ]),
            self.datensatz_dropdown,
            widgets.HTML('<hr>'),
            widgets.HTML('<b>Messwerte eingeben:</b>'),
            self.spannung_input,
            self.stromstaerke_input,
            self.drehzahl_input,
            self.notiz_input,
            widgets.HBox([
                self.speichern_button,
                self.loeschen_wert_button
            ]),
            widgets.HTML('<hr>'),
            widgets.HTML('<b>Datensatz-Aktionen:</b>'),
            widgets.HBox([
                self.anzeigen_button,
                self.alle_anzeigen_button,
                self.loeschen_datensatz_button
            ]),
            widgets.HBox([
                self.export_button,
                self.export_alle_button
            ]),
            widgets.HTML('<hr>'),
            widgets.HTML('<b>Session-Verwaltung:</b>'),
            widgets.HBox([
                self.save_session_button,
                self.load_session_button
            ]),
            self.output
        ])
    
    def parse_float(self, value_str):
        """Konvertiert String zu Float mit Fehlerbehandlung"""
        try:
            # Ersetze Komma durch Punkt für deutsche Zahlenformate
            value_str = value_str.replace(',', '.')
            return float(value_str)
        except ValueError:
            return 0.0
        
    def neuer_datensatz(self, b):
        """Erstellt einen neuen Datensatz"""
        with self.output:
            clear_output()
            
            name = self.datensatz_name.value.strip()
            if not name:
                print("❌ Bitte geben Sie einen Namen für den Datensatz ein!")
                return
            
            if name in self.alle_datensaetze:
                print(f"❌ Ein Datensatz mit dem Namen '{name}' existiert bereits!")
                return
            
            # Neuen Datensatz erstellen
            self.alle_datensaetze[name] = {
                'info': {
                    'name': name,
                    'erstellt': datetime.now().strftime('%Y-%m-%d %H:%M:%S'),
                    'anzahl_messungen': 0
                },
                'messungen': []
            }
            
            self.aktueller_datensatz = name
            self.datensatz_name.value = ''
            
            # Dropdown aktualisieren
            optionen = ['-- Neuer Datensatz --'] + list(self.alle_datensaetze.keys())
            self.datensatz_dropdown.options = optionen
            self.datensatz_dropdown.value = name
            
            # Buttons aktivieren
            self.speichern_button.disabled = False
            
            print(f"✓ Datensatz '{name}' wurde erstellt!")
    
    def datensatz_wechseln(self, change):
        """Wechselt zwischen Datensätzen"""
        with self.output:
            clear_output()
            
            if change['new'] == '-- Neuer Datensatz --':
                self.aktueller_datensatz = None
                self.speichern_button.disabled = True
                print("Geben Sie einen Namen für einen neuen Datensatz ein.")
            else:
                self.aktueller_datensatz = change['new']
                self.speichern_button.disabled = False
                info = self.alle_datensaetze[self.aktueller_datensatz]['info']
                print(f"Aktiver Datensatz: '{self.aktueller_datensatz}'")
                print(f"Erstellt: {info['erstellt']}")
                print(f"Anzahl Messungen: {info['anzahl_messungen']}")
    
    def speichern_wert(self, b):
        """Speichert einen Messwert im aktuellen Datensatz"""
        with self.output:
            clear_output()
            
            if not self.aktueller_datensatz:
                print("❌ Bitte erstellen Sie zuerst einen Datensatz!")
                return
            
            # Werte auslesen und konvertieren
            spannung = self.parse_float(self.spannung_input.value)
            stromstaerke = self.parse_float(self.stromstaerke_input.value)
            drehzahl = self.parse_float(self.drehzahl_input.value)
            notiz = self.notiz_input.value
            
            # Validierung
            if spannung == 0 and stromstaerke == 0 and drehzahl == 0:
                print("❌ Bitte geben Sie mindestens einen Wert ein!")
                return
            
            # Messung hinzufügen
            messung = {
                'Zeitstempel': datetime.now().strftime('%Y-%m-%d %H:%M:%S'),
                'Spannung (V)': spannung,
                'Stromstärke (mA)': stromstaerke,
                'Drehzahl (U/min)': drehzahl,
                'Leistung (mW)': round(spannung * stromstaerke, 2),
                'Notiz': notiz
            }
            
            self.alle_datensaetze[self.aktueller_datensatz]['messungen'].append(messung)
            self.alle_datensaetze[self.aktueller_datensatz]['info']['anzahl_messungen'] += 1
            
            print(f"✓ Messung zu '{self.aktueller_datensatz}' hinzugefügt!")
            print(f"  Spannung: {spannung} V")
            print(f"  Stromstärke: {stromstaerke} mA")
            print(f"  Drehzahl: {drehzahl} U/min")
            print(f"  Leistung: {round(spannung * stromstaerke, 2)} mW")
            if notiz:
                print(f"  Notiz: {notiz}")
            print(f"\nAnzahl Messungen in diesem Datensatz: {self.alle_datensaetze[self.aktueller_datensatz]['info']['anzahl_messungen']}")
            
            # Eingabefelder zurücksetzen
            self.spannung_input.value = ''
            self.stromstaerke_input.value = ''
            self.drehzahl_input.value = ''
            self.notiz_input.value = ''
    
    def letzten_wert_loeschen(self, b):
        """Löscht den letzten Wert aus dem aktuellen Datensatz"""
        with self.output:
            clear_output()
            
            if not self.aktueller_datensatz:
                print("❌ Kein Datensatz ausgewählt!")
                return
            
            messungen = self.alle_datensaetze[self.aktueller_datensatz]['messungen']
            if messungen:
                gelöscht = messungen.pop()
                self.alle_datensaetze[self.aktueller_datensatz]['info']['anzahl_messungen'] -= 1
                print(f"✓ Letzte Messung aus '{self.aktueller_datensatz}' gelöscht:")
                print(f"  Zeitstempel: {gelöscht['Zeitstempel']}")
                print(f"  Werte: {gelöscht['Spannung (V)']}V, {gelöscht['Stromstärke (mA)']}mA, {gelöscht['Drehzahl (U/min)']}U/min")
            else:
                print("❌ Keine Messungen zum Löschen vorhanden!")
    
    def anzeigen_aktueller_datensatz(self, b):
        """Zeigt den aktuellen Datensatz an"""
        with self.output:
            clear_output()
            
            if not self.aktueller_datensatz:
                print("❌ Kein Datensatz ausgewählt!")
                return
            
            messungen = self.alle_datensaetze[self.aktueller_datensatz]['messungen']
            if not messungen:
                print(f"Datensatz '{self.aktueller_datensatz}' enthält keine Messungen!")
            else:
                df = pd.DataFrame(messungen)
                print(f"=== Datensatz: {self.aktueller_datensatz} ===")
                display(df)
                
                # Statistiken
                print("\n--- Statistiken ---")
                print(f"Anzahl Messungen: {len(df)}")
                print(f"Durchschnittliche Spannung: {df['Spannung (V)'].mean():.2f} V")
                print(f"Durchschnittliche Stromstärke: {df['Stromstärke (mA)'].mean():.2f} mA")
                print(f"Durchschnittliche Drehzahl: {df['Drehzahl (U/min)'].mean():.2f} U/min")
                print(f"Durchschnittliche Leistung: {df['Leistung (mW)'].mean():.2f} mW")
                print(f"Maximale Leistung: {df['Leistung (mW)'].max():.2f} mW")
                print(f"Minimale Leistung: {df['Leistung (mW)'].min():.2f} mW")
    
    def alle_anzeigen(self, b):
        """Zeigt eine Übersicht aller Datensätze"""
        with self.output:
            clear_output()
            
            if not self.alle_datensaetze:
                print("Keine Datensätze vorhanden!")
                return
            
            print("=== Übersicht aller Datensätze ===\n")
            for name, daten in self.alle_datensaetze.items():
                info = daten['info']
                print(f"📊 {name}")
                print(f"   Erstellt: {info['erstellt']}")
                print(f"   Anzahl Messungen: {info['anzahl_messungen']}")
                
                if daten['messungen']:
                    df = pd.DataFrame(daten['messungen'])
                    print(f"   Durchschn. Leistung: {df['Leistung (mW)'].mean():.2f} mW")
                print()
    
    def datensatz_loeschen(self, b):
        """Löscht den aktuellen Datensatz"""
        with self.output:
            clear_output()
            
            if not self.aktueller_datensatz:
                print("❌ Kein Datensatz ausgewählt!")
                return
            
            # Bestätigung anzeigen
            print(f"⚠️  Möchten Sie den Datensatz '{self.aktueller_datensatz}' wirklich löschen?")
            print(f"   Enthält {self.alle_datensaetze[self.aktueller_datensatz]['info']['anzahl_messungen']} Messungen")
            
            # Löschen
            del self.alle_datensaetze[self.aktueller_datensatz]
            
            # Dropdown aktualisieren
            optionen = ['-- Neuer Datensatz --'] + list(self.alle_datensaetze.keys())
            self.datensatz_dropdown.options = optionen
            self.datensatz_dropdown.value = '-- Neuer Datensatz --'
            
            print(f"\n✓ Datensatz '{self.aktueller_datensatz}' wurde gelöscht!")
            self.aktueller_datensatz = None
    
    def export_aktueller_datensatz(self, b):
        """Exportiert den aktuellen Datensatz als CSV"""
        with self.output:
            clear_output()
            
            if not self.aktueller_datensatz:
                print("❌ Kein Datensatz ausgewählt!")
                return
            
            messungen = self.alle_datensaetze[self.aktueller_datensatz]['messungen']
            if not messungen:
                print("❌ Keine Messungen zum Exportieren vorhanden!")
                return
            
            df = pd.DataFrame(messungen)
            filename = f"{self.aktueller_datensatz}_{datetime.now().strftime('%Y%m%d_%H%M%S')}.csv"
            df.to_csv(filename, index=False, encoding='utf-8')
            print(f"✓ Datensatz '{self.aktueller_datensatz}' wurde als '{filename}' exportiert!")
    
    def export_alle(self, b):
        """Exportiert alle Datensätze als separate CSV-Dateien"""
        with self.output:
            clear_output()
            
            if not self.alle_datensaetze:
                print("❌ Keine Datensätze zum Exportieren vorhanden!")
                return
            
            timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
            exported = 0
            
            for name, daten in self.alle_datensaetze.items():
                if daten['messungen']:
                    df = pd.DataFrame(daten['messungen'])
                    filename = f"{name}_{timestamp}.csv"
                    df.to_csv(filename, index=False, encoding='utf-8')
                    exported += 1
                    print(f"✓ '{name}' exportiert als '{filename}'")
            
            print(f"\n✓ {exported} Datensätze exportiert!")
    
    def save_session(self, b):
        """Speichert alle Datensätze in einer JSON-Datei"""
        with self.output:
            clear_output()
            
            if not self.alle_datensaetze:
                print("❌ Keine Datensätze zum Speichern vorhanden!")
                return
            
            filename = f"session_{datetime.now().strftime('%Y%m%d_%H%M%S')}.json"
            with open(filename, 'w', encoding='utf-8') as f:
                json.dump(self.alle_datensaetze, f, ensure_ascii=False, indent=2)
            
            print(f"✓ Session gespeichert als '{filename}'")
            print(f"  Enthält {len(self.alle_datensaetze)} Datensätze")
    
    def load_session(self, b):
        """Lädt Datensätze aus einer JSON-Datei"""
        with self.output:
            clear_output()
            
            # File-Upload Widget
            upload = widgets.FileUpload(
                accept='.json',
                multiple=False,
                description='JSON laden:'
            )
            
            def on_upload(change):
                content = upload.value[0]['content']
                content_str = content.tobytes().decode('utf-8')
                
                try:
                    loaded_data = json.loads(content_str)
                    self.alle_datensaetze = loaded_data
                    
                    # Dropdown aktualisieren
                    optionen = ['-- Neuer Datensatz --'] + list(self.alle_datensaetze.keys())
                    self.datensatz_dropdown.options = optionen
                    
                    print(f"✓ Session geladen!")
                    print(f"  {len(self.alle_datensaetze)} Datensätze importiert")
                    
                except Exception as e:
                    print(f"❌ Fehler beim Laden: {e}")
            
            upload.observe(on_upload, names='value')
            display(upload)
            print("Wählen Sie eine JSON-Datei zum Laden aus...")
    
    def display(self):
        """Zeigt die Eingabemaske an"""
        display(self.layout)

# Verwendung:
eingabe = MehrfachDatenEingabe()
eingabe.display()